## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,28.06,100,75,8.05,broken clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,33.91,69,75,6.71,broken clouds
2,2,Marawi,PH,7.9986,124.2928,67.64,91,100,2.55,overcast clouds
3,3,Cootamundra,AU,-34.6500,148.0333,48.94,54,0,3.29,clear sky
4,4,Upernavik,GL,72.7868,-56.1549,31.60,93,100,4.54,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Atuona,PF,-9.8000,-139.0333,79.56,64,6,9.26,clear sky
14,14,Coahuayana,MX,18.7333,-103.6833,86.86,48,83,5.64,broken clouds
19,19,Hithadhoo,MV,-0.6000,73.0833,83.71,71,14,18.79,few clouds
21,21,Faanui,PF,-16.4833,-151.7500,79.84,73,99,16.11,overcast clouds
23,23,Avarua,CK,-21.2078,-159.7750,78.85,57,52,11.50,broken clouds
26,26,Trairi,BR,-3.2778,-39.2689,77.45,83,4,8.70,clear sky
30,30,Rikitea,PF,-23.1203,-134.9692,75.34,67,88,12.26,overcast clouds
31,31,Ishigaki,JP,24.3448,124.1572,80.73,94,75,4.61,broken clouds
36,36,Vung Tau,VN,10.3460,107.0843,82.40,81,48,9.64,scattered clouds
38,38,Morant Bay,JM,17.8815,-76.4093,83.82,80,66,7.81,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                232
City                   232
Country                232
Lat                    232
Lng                    232
Max Temp               232
Humidity               232
Cloudiness             232
Wind Speed             232
Current Description    232
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_df.count()

City_ID                232
City                   232
Country                232
Lat                    232
Lng                    232
Max Temp               232
Humidity               232
Cloudiness             232
Wind Speed             232
Current Description    232
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Atuona,PF,79.56,clear sky,-9.8000,-139.0333,
14,Coahuayana,MX,86.86,broken clouds,18.7333,-103.6833,
19,Hithadhoo,MV,83.71,few clouds,-0.6000,73.0833,
21,Faanui,PF,79.84,overcast clouds,-16.4833,-151.7500,
23,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,
26,Trairi,BR,77.45,clear sky,-3.2778,-39.2689,
30,Rikitea,PF,75.34,overcast clouds,-23.1203,-134.9692,
31,Ishigaki,JP,80.73,broken clouds,24.3448,124.1572,
36,Vung Tau,VN,82.40,scattered clouds,10.3460,107.0843,
38,Morant Bay,JM,83.82,broken clouds,17.8815,-76.4093,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

SyntaxError: invalid syntax (<ipython-input-15-169a513dc833>, line 11)

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Atuona,PF,79.56,clear sky,-9.8000,-139.0333,
14,Coahuayana,MX,86.86,broken clouds,18.7333,-103.6833,
19,Hithadhoo,MV,83.71,few clouds,-0.6000,73.0833,
21,Faanui,PF,79.84,overcast clouds,-16.4833,-151.7500,
23,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="Hotel_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))